In [45]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [46]:
path = '../data/'
savepath = '../data/'
images = glob.glob(path+'new_data/*.npy')

In [47]:
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
tile_sizes = tile_sizes[:150]
images_sampled = {}
for tile in tile_sizes:
    for i in range(1, 31):
        images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])

In [53]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            temp  = imresize(frame[tile_index], (24, 24))
            temp  = imresize(temp, (384, 384))
            
            alldata_x.append(temp)
            alldata_y.append(frame[tile_index])
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [54]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)


        conv1 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = LeakyReLU()(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)



        conv2 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv2 = LeakyReLU()(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv3 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv3 = LeakyReLU()(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        concat1 = add([conv1, conv3])

        conv4 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(concat1)
        conv4 = LeakyReLU()(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        conv5 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv5 = LeakyReLU()(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        concat2 = add([conv5, concat1])

        conv6 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(concat2)
        conv6 = LeakyReLU()(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        conv7 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv7 = LeakyReLU()(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        concat3 = add([conv7, concat2])

        conv8 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        conv8 = LeakyReLU()(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        conv9 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = LeakyReLU()(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        

        conv10 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU()(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)

        concat4 = add([conv10, concat3])
        
        conv11 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(concat4)
        conv11 = LeakyReLU()(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)
        
        conv12 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv11)
        conv12 = LeakyReLU()(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)

        concat5 = add([conv12, conv1])
        
        conv13 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(concat5)
        conv13 = LeakyReLU()(conv13)
        conv13 = BatchNormalization(momentum=0.8)(conv13)
        
        conv14 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv13)
        conv14 = LeakyReLU()(conv14)
        conv14 = BatchNormalization(momentum=0.8)(conv14)

        out = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv14)
        out = LeakyReLU()(out)
        out = BatchNormalization(momentum=0.8)(out)


        model = Model(input = inputs, output = out)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, y_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        y_train = (0.5 * y_train + 0.5)*255
        y_train = y_train.astype(int)
        X_train = X_train*255
        X_train = X_train.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5], y_train[6], y_train[7], y_train[8], y_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_lowres.png", combined)

In [ ]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(200000, 10, 100)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:119: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv2d_127 (Conv2D)             (None, 384, 384, 16) 448         input_21[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_113 (LeakyReLU)     (None, 384, 384, 16) 0           conv2d_127[0][0]                 
__________________________________________________________________________________________________
batch_normalization_127 (BatchN (None, 384, 384, 16) 64          leaky_re_lu_113[0][0]            
__________________________________________________________________________________________________
conv2d_128

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':


Epoch  0  G loss  1.3565364


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:149: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:153: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:157: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


...

Epoch  31884  G loss  0.022316355
Epoch  31885  G loss  0.06010022
Epoch  31886  G loss  0.04621781
Epoch  31887  G loss  0.012368626
Epoch  31888  G loss  0.011595149
Epoch  31889  G loss  0.016560549
Epoch  31890  G loss  0.012868142
Epoch  31891  G loss  0.014337689
Epoch  31892  G loss  0.13952217
Epoch  31893  G loss  0.019139325
Epoch  31894  G loss  0.045550417
Epoch  31895  G loss  0.037042916
Epoch  31896  G loss  0.04278595
Epoch  31897  G loss  0.03269669
Epoch  31898  G loss  0.07203003
Epoch  31899  G loss  0.024675123
Epoch  31900  G loss  0.031979125
Epoch  31901  G loss  0.025859678
Epoch  31902  G loss  0.16576993
Epoch  31903  G loss  0.105600454
Epoch  31904  G loss  0.013493854
Epoch  31905  G loss  0.052700948
Epoch  31906  G loss  0.013815977
Epoch  31907  G loss  0.040186655
Epoch  31908  G loss  0.05294188
Epoch  31909  G loss  0.025292328
Epoch  31910  G loss  0.07683054
Epoch  31911  G loss  0.02663126
Epoch  31912  G loss  0.030479869
Epoch  31913  G loss  0.

Epoch  32128  G loss  0.02004956
Epoch  32129  G loss  0.013810763
Epoch  32130  G loss  0.021501232
Epoch  32131  G loss  0.019373406
Epoch  32132  G loss  0.025024468
Epoch  32133  G loss  0.032641836
Epoch  32134  G loss  0.03677919
Epoch  32135  G loss  0.028352892
Epoch  32136  G loss  0.019207712
Epoch  32137  G loss  0.13748585
Epoch  32138  G loss  0.018794388
Epoch  32139  G loss  0.03533865
Epoch  32140  G loss  0.044001717
Epoch  32141  G loss  0.022470335
Epoch  32142  G loss  0.0703678
Epoch  32143  G loss  0.049368884
Epoch  32144  G loss  0.06616689
Epoch  32145  G loss  0.055442654
Epoch  32146  G loss  0.101223804
Epoch  32147  G loss  0.03925421
Epoch  32148  G loss  0.06502839
Epoch  32149  G loss  0.03396064
Epoch  32150  G loss  0.021383598
Epoch  32151  G loss  0.01620798
Epoch  32152  G loss  0.0727001
Epoch  32153  G loss  0.020214671
Epoch  32154  G loss  0.057342064
Epoch  32155  G loss  0.0221319
Epoch  32156  G loss  0.024937117
Epoch  32157  G loss  0.05204

Epoch  32372  G loss  0.017229054
Epoch  32373  G loss  0.08734564
Epoch  32374  G loss  0.0453809
Epoch  32375  G loss  0.014812082
Epoch  32376  G loss  0.04528857
Epoch  32377  G loss  0.089273624
Epoch  32378  G loss  0.026546681
Epoch  32379  G loss  0.089899965
Epoch  32380  G loss  0.01355044
Epoch  32381  G loss  0.04116288
Epoch  32382  G loss  0.020219164
Epoch  32383  G loss  0.026306605
Epoch  32384  G loss  0.09575324
Epoch  32385  G loss  0.022529315
Epoch  32386  G loss  0.065002576
Epoch  32387  G loss  0.019865591
Epoch  32388  G loss  0.10471137
Epoch  32389  G loss  0.041244913
Epoch  32390  G loss  0.048197757
Epoch  32391  G loss  0.0339282
Epoch  32392  G loss  0.028145287
Epoch  32393  G loss  0.043752518
Epoch  32394  G loss  0.05791111
Epoch  32395  G loss  0.018169736
Epoch  32396  G loss  0.036217146
Epoch  32397  G loss  0.02479717
Epoch  32398  G loss  0.016338559
Epoch  32399  G loss  0.042748652
Epoch  32400  G loss  0.10138025
Epoch  32401  G loss  0.019

Epoch  32617  G loss  0.016709197
Epoch  32618  G loss  0.09631072
Epoch  32619  G loss  0.020164218
Epoch  32620  G loss  0.057030033
Epoch  32621  G loss  0.024080468
Epoch  32622  G loss  0.07127857
Epoch  32623  G loss  0.039939098
Epoch  32624  G loss  0.025586814
Epoch  32625  G loss  0.13816419
Epoch  32626  G loss  0.024367366
Epoch  32627  G loss  0.026981086
Epoch  32628  G loss  0.022196662
Epoch  32629  G loss  0.027201131
Epoch  32630  G loss  0.026937902
Epoch  32631  G loss  0.03937486
Epoch  32632  G loss  0.014461769
Epoch  32633  G loss  0.03260705
Epoch  32634  G loss  0.0511536
Epoch  32635  G loss  0.019149996
Epoch  32636  G loss  0.024962083
Epoch  32637  G loss  0.12075609
Epoch  32638  G loss  0.0219124
Epoch  32639  G loss  0.050664723
Epoch  32640  G loss  0.061488915
Epoch  32641  G loss  0.033188008
Epoch  32642  G loss  0.06539844
Epoch  32643  G loss  0.016706329
Epoch  32644  G loss  0.12935297
Epoch  32645  G loss  0.12975986
Epoch  32646  G loss  0.051

Epoch  32861  G loss  0.018704932
Epoch  32862  G loss  0.08422056
Epoch  32863  G loss  0.033633035
Epoch  32864  G loss  0.14995743
Epoch  32865  G loss  0.024734095
Epoch  32866  G loss  0.13022979
Epoch  32867  G loss  0.05362184
Epoch  32868  G loss  0.11250074
Epoch  32869  G loss  0.03233906
Epoch  32870  G loss  0.015129572
Epoch  32871  G loss  0.05371765
Epoch  32872  G loss  0.06502925
Epoch  32873  G loss  0.031307843
Epoch  32874  G loss  0.014591691
Epoch  32875  G loss  0.050440352
Epoch  32876  G loss  0.13242139
Epoch  32877  G loss  0.022753809
Epoch  32878  G loss  0.05695319
Epoch  32879  G loss  0.028900873
Epoch  32880  G loss  0.020314302
Epoch  32881  G loss  0.017664274
Epoch  32882  G loss  0.10338044
Epoch  32883  G loss  0.022952065
Epoch  32884  G loss  0.029015755
Epoch  32885  G loss  0.02237185
Epoch  32886  G loss  0.013131837
Epoch  32887  G loss  0.035837345
Epoch  32888  G loss  0.050195742
Epoch  32889  G loss  0.036537044
Epoch  32890  G loss  0.02

In [31]:
for i in range(1000):
    x = myGenerator(1)
    xtest, ytest = next(x)
    pred = cgan.generator.predict(xtest)
    pred = pred*127.5 + 127.5
    pred = pred.astype(int)
    #plt.imshow(pred[0])
    #plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    #plt.imshow(ytest[0])
    #plt.show()
    imsave(path+'test/frame_pred'+str(i)+'.png', pred[0])
    imsave(path+'test/frame_real'+str(i)+'.png', ytest[0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [32]:
import numpy 
import math
import cv2

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [36]:
data = []
for i in range(1000):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = cv2.imread(path+'test/frame_pred'+str(i)+'.png')
    d = psnr(img1, img2)
    data.append(d)
print('PSNR:', np.mean(data))

PSNR: 31.612699630864867
